In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL").config("spark.sql.catalogImplementation" , "hive").config("spark.hadoop.native.lib", "false").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").enableHiveSupport().getOrCreate()

In [2]:
#reading data
csv_file = "data/departuredelays.csv"

df = (spark.read.format("csv").
     option("inferSchema", "true")
     .option("header","true")
     .load(csv_file))
df.createOrReplaceTempView("us_delay_flights_tbl")

In [5]:
spark.sql("""SELECT distance, origin, destination
FROM us_delay_flights_tbl WHERE distance > 1000
ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [6]:
spark.sql("""SELECT delay, origin, destination,
CASE
WHEN delay > 360 THEN 'Very Long Delays'
WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
WHEN delay = 0 THEN 'No Delays'
ELSE 'Early'
END AS Flight_Delays
FROM us_delay_flights_tbl
ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [5]:
from pyspark.sql.functions import col, desc
(df.select("distance", "origin", "destination")
.where(col("distance") > 1000)
.orderBy(desc("distance"))).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [6]:
from pyspark.sql.functions import when,col, desc 
(df.select(
        "delay", 
        "origin", 
        "destination",
        when(col("delay") > 360, 'Very Long Delays')
            .when((col("delay") > 120) & (col("delay") < 360), 'Long Delays')
            .when((col("delay") > 60) & (col("delay") < 120), 'Short Delays')
            .when((col("delay") > 0) & (col("delay") < 60), 'Tolerable Delays')
            .when(col("delay") == 0, 'No Delays')
            .otherwise('Early')
            .alias("Flight_Delays")
    )
    .orderBy(col("origin"), desc("delay"))).show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [7]:
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

AnalysisException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `learn_spark_db` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.

In [10]:
schema="date STRING, delay INT, distance INT, origin STRING, destination STRING"
flights_df = spark.read.csv(csv_file, schema=schema)
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `spark_catalog`.`default`.`managed_us_delay_flights_tbl` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

In [ ]:
# alternative way to create table
spark.sql("CREATE TABLE managed_us_delay_flights_tbl (date STRING, delay INT,distance INT, origin STRING, destination STRING)")

In [9]:
(flights_df
.write
.option("path", "/data/departuredelays")
.saveAsTable("us_delay_flights_tbl"))

NameError: name 'flights_df' is not defined

In [11]:
flights_df

DataFrame[date: string, delay: int, distance: int, origin: string, destination: string]

In [12]:
# In Python
df_sfo = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")
df_jfk = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'JFK'")
# Create a temporary and global temporary view
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")


In [13]:
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view")

DataFrame[date: int, delay: int, origin: string, destination: string]

In [14]:
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

True

In [15]:
spark.catalog.listDatabases()
spark.catalog.listTables()
# spark.catalog.listColumns("us_delay_flights_tbl")

[Table(name='managed_us_delay_flights_tbl', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='us_delay_flights_tbl', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='us_delay_flights_tbl', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

### Parquet

In [16]:
file = "data/part-r-00000-1a9822ba-b8fb-4d8e-844a-ea30d0801b9e.gz.parquet"
df = spark.read.format("parquet").load(file)

In [17]:
(df.write.format("parquet")
.mode("overwrite")
.option("compression", "snappy")
.save("tmp/data/parquet/df_parquet"))

In [18]:
(df.write
.mode("overwrite")
.saveAsTable("us_delay_flights_tbl"))

### CSV

In [ ]:
file = "data/csv/*"

schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"
df = (spark.read.format("csv")
.option("header", "true")
.schema(schema)
.option("mode", "FAILFAST") # Exit if any errors
.option("nullValue", "") # Replace any null data field with quotes
.load(file))

In [ ]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl_csv
USING csv
OPTIONS (
path "data/csv/*",
header "true",
inferSchema "true",
mode "FAILFAST"
)""")

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl_csv").show(10)

In [ ]:
df.write.format("csv").mode("overwrite").save("tmp/data/csv/df_csv")

### User Defined Functions

In [9]:
from pyspark.sql.types import LongType
# Create cubed function
def cubed(s):
    return s * s * s
# Register UDF
spark.udf.register("cubed", cubed, LongType())
# Generate temporary view
spark.range(1, 9).createOrReplaceTempView("udf_test")

In [11]:
spark.sql("SELECT id, cubed(id) AS id_cubed FROM udf_test").show()

Py4JJavaError: An error occurred while calling o60.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 4.0 failed 1 times, most recent failure: Lost task 1.0 in stage 4.0 (TID 26) (192.168.60.248 executor driver): java.net.SocketException: Connection reset
	at java.base/sun.nio.ch.NioSocketImpl.implRead(NioSocketImpl.java:323)
	at java.base/sun.nio.ch.NioSocketImpl.read(NioSocketImpl.java:350)
	at java.base/sun.nio.ch.NioSocketImpl$1.read(NioSocketImpl.java:803)
	at java.base/java.net.Socket$SocketInputStream.read(Socket.java:966)
	at java.base/java.io.BufferedInputStream.fill(BufferedInputStream.java:244)
	at java.base/java.io.BufferedInputStream.read(BufferedInputStream.java:263)
	at java.base/java.io.DataInputStream.readInt(DataInputStream.java:393)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:83)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonU